# Chapter17.2_Practice
<b> Naiema Elsaadi</b>

# 17.2 Relational Databases and Structured Query Language (SQL)

<b>Tables, Rows and Columns  / Selecting Data Subsets  / SQLite 

# 17.2.1 A books Database

We will present a books database containing information about several of our books.

<b> Creating the books Database </b>


sqlite3 books.db < books.sql

<b> Connecting to the Database in Python</b>

In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('books.db')

<b>authors Table</b>
<br>id 
<br>first
<br>last

<b> Viewing the authors Table's Contents</b> 

In [3]:
import pandas as pd

In [4]:
pd.options.display.max_columns = 10

In [5]:
pd.read_sql('SELECT * FROM authors', connection,
            index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


<b> titles Table</b>

In [6]:
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


<b>author_ISBN Table</b>

In [7]:
df = pd.read_sql('SELECT * FROM author_ISBN', connection)

In [8]:
df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


<b> Entity-Relationship(ER) Diagram</b>

<b> SQL Keywords</b>

# 17.2.2 SELECT Queries

In [9]:
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


# 17.2.3 WHERE Clause

In [10]:
pd.read_sql("""SELECT title, edition, copyright 
                FROM titles 
                WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


<b>Pattern Matching: Zero or More Characters</b>

In [11]:
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                WHERE last LIKE 'D%'""", 
             connection, index_col=['id'])


,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


<b> Pattern Matching: Any Character</b>

In [12]:
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                WHERE first LIKE '_b%'""", 
             connection, index_col=['id'])


,first,last
id,,
3,Abbey,Deitel


# 17.2.4 ORDER BY Clause

In [13]:
pd.read_sql('SELECT title FROM titles ORDER BY title ASC',
             connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


<b> Sorting By Multiple Columns</b>

In [14]:
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                ORDER BY last, first""", 
             connection, index_col=['id'])


,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [15]:
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                ORDER BY last DESC, first ASC""", 
             connection, index_col=['id'])


,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


<b>Combining the WHERE and ORDER BY Clauses</b>

In [16]:
pd.read_sql("""SELECT isbn, title, edition, copyright
                FROM titles
                WHERE title LIKE '%How to Program'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


# 17.2.5 Merging Data from Multiple Tables: INNER JOIN

In [17]:
pd.read_sql("""SELECT first, last, isbn
                FROM authors
                INNER JOIN author_ISBN
                    ON authors.id = author_ISBN.id
                ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


# 17.2.6 INSERT INTO Statement

Use a sqlite3 cursor object

In [18]:
cursor = connection.cursor()

In [19]:
cursor = cursor.execute("""INSERT INTO authors (first, last)
                            VALUES ('Sue', 'Red')""")

In [20]:
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


# 17.2.7 UPDATE Statement

In [21]:
cursor = cursor.execute("""UPDATE authors SET last='Black'
                            WHERE last='Red' AND first='Sue'""") 

In [22]:
cursor.rowcount

1

In [23]:
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


# 17.2.8 DELETE FROM Statement

In [24]:
cursor = cursor.execute('DELETE FROM authors where id=6') 

In [25]:
cursor.rowcount

1

In [26]:
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


<b>Closing the Datebase</b>
<br>When you no longer need access to the database, you should call the Connection's close method to disconnect from the database<b> not yet</b> though, as I will use the database in the next self check exercises. I will use the<b> connection.close()</b> command in the end when I am finished.

<b> SQL in Big Data</b>
<br> SQl's importance and usage is growing in big data. Later in this chapter, we will use Spark SQL to query data in a Spark DataFrame which will distribute the data across many computers in a Spark cluster. As you will see, Spark SQl looks very similar to the SQL presented in this sction. Spark SQl will also be used in the excerises.

# Self Check for Section 17.2

<b> Show only the first three results.<b>

In [27]:
pd.read_sql("""SELECT title, edition FROM titles ORDER BY edition DESC""", connection).head(3)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8


<b>Select from authors table all authors whose first name starts whit 'A'</b>

In [28]:
pd.read_sql("""SELECT * FROM authors WHERE first LIKE 'A%'""", connection)

,id,first,last
0,3,Abbey,Deitel
1,5,Alexander,Wald


<b>Select from the table all titles that do not end with 'How to Program'.</b>

In [29]:
pd.read_sql("""SELECT isbn, title, edition, copyright
                FROM titles
                WHERE title NOT LIKE '%How to Program'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134289366,Android 6 for Programmers,3,2016
1,0135404673,Intro to Python for CS and DS,1,2020


<b>Close the Database</b>

In [30]:
connection.close()

In [31]:
print('My name is Naiema Elsaadi')

My name is Naiema Elsaadi
